# Positional Burrows-Wheeler Transform (pbwt) Demo

In [1]:
X = [[0, 1, 0, 1, 0, 1],
     [1, 1, 0, 0, 0, 1],
     [1, 1, 1, 1, 1, 1],
     [0, 1, 1, 1, 1, 0],
     [0, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 1, 0],
     [1, 1, 0, 0, 0, 1],
     [0, 1, 0, 1, 1, 0]]

In [2]:
def build_prefix_array(X):
    
    # M haplotypes
    M = len(X)

    # N variable sites
    N = len(X[0])
    
    # initialise positional prefix array
    ppa = list(range(M))
    
    # iterate over variants
    for k in range(N-1):

        # setup intermediates
        a = list()
        b = list()
    
        # iterate over haplotypes in reverse prefix sorted order
        for index in ppa:

            # current haplotype
            haplotype = X[index]
            
            # allele for current haplotype
            allele = haplotype[k]
            
            # update intermediates
            if allele == 0:
                a.append(index)
            else:
                b.append(index)
                
        # construct the new positional prefix array for k+1 by concatenating lists a and b
        ppa = a + b
        
    return ppa


In [3]:
build_prefix_array(X)

[4, 1, 6, 0, 5, 7, 3, 2]

In [4]:
def display_prefix_array(X):
    from IPython.display import display_html
    ppa = build_prefix_array(X)
    html = '<pre style="line-height: 100%">'
    for index in ppa:
        haplotype = X[index]
        html += str(index) + '|' + ''.join(str(allele) for allele in haplotype[:-1])
        html += '  ' + str(haplotype[-1]) + '<br/>'
    html += '</pre>'
    display_html(html, raw=True)

In [5]:
display_prefix_array(X)

4|00000 0 1|11000 1 6|11000 1 0|01010 1 5|10001 0 7|01011 0 3|01111 0 2|11111 1

In [6]:
def build_prefix_and_divergence_arrays(X):
    
    # M haplotypes
    M = len(X)

    # N variable sites
    N = len(X[0])
    
    # initialise positional prefix array
    ppa = list(range(M))
    
    # initialise divergence array
    div = [0] * M
    
    # iterate over variants
    for k in range(N-1):

        # setup intermediates
        a = list()
        b = list()
        d = list()
        e = list()
        p = q = k + 1
    
        # iterate over haplotypes in reverse prefix sorted order
        for index, match_start in zip(ppa, div):

            # current haplotype
            haplotype = X[index]
            
            # allele for current haplotype
            allele = haplotype[k]
            
            # update intermediates
            if match_start > p:
                p = match_start
            if match_start > q:
                q = match_start

            # update intermediates
            if allele == 0:
                a.append(index)
                d.append(p)
                p = 0
            else:
                b.append(index)
                e.append(q)
                q = 0
                
        # construct the new arrays for k+1 by concatenating intermediates
        ppa = a + b
        div = d + e
        
    return ppa, div

In [7]:
ppa, div = build_prefix_and_divergence_arrays(X)

In [8]:
ppa

[4, 1, 6, 0, 5, 7, 3, 2]

In [9]:
div

[5, 2, 0, 4, 5, 4, 3, 1]

In [10]:
def display_prefix_and_divergence_arrays(X):
    from IPython.display import display_html
    ppa, div = build_prefix_and_divergence_arrays(X)
    html = '<pre style="line-height: 100%">'
    for index, match_start in zip(ppa, div):
        haplotype = X[index]
        html += str(index) + '|'
        for k, allele in enumerate(haplotype[:-1]):
            if match_start == k:
                html += '<strong><u>'
            html += str(allele)
        if match_start < len(haplotype) - 1:
            html += '</u></strong>'
        html += '  ' + str(haplotype[-1]) + '<br/>'
    html += '</pre>'
    display_html(html, raw=True)

In [11]:
display_prefix_and_divergence_arrays(X)

4|00000 0 1|11 000 1 6| 11000 1 0|0101 0 1 5|10001 0 7|0101 1 0 3|011 11 0 2|1 1111 1

In [12]:
def report_long_matches(X, L):
    
    # M haplotypes
    M = len(X)

    # N variable sites
    N = len(X[0])
    
    # initialise positional prefix array
    ppa = list(range(M))
    
    # initialise divergence array
    div = [0] * M
    
    # iterate over variants
    for k in range(N):

        # setup intermediates
        a = list()
        b = list()
        d = list()
        e = list()
        p = q = k + 1
        ma = list()
        mb = list()
    
        # iterate over haplotypes in reverse prefix sorted order
        for index, match_start in zip(ppa, div):
            
            # report matches
            if match_start > k - L:
                if ma and mb:
                    yield k, ma, mb
                ma = list()
                mb = list()

            # current haplotype
            haplotype = X[index]
            
            # allele for current haplotype
            allele = haplotype[k]
            
            # update intermediates
            if match_start > p:
                p = match_start
            if match_start > q:
                q = match_start

            # update intermediates
            if allele == 0:
                a.append(index)
                d.append(p)
                p = 0
                ma.append(index)
            else:
                b.append(index)
                e.append(q)
                q = 0
                mb.append(index)
                
        # report any remaining matches including final haplotype (N.B., not in Durbin 2014)
        if ma and mb:
            yield k, ma, mb
                
        # construct the new arrays for k+1
        if k < N - 1:
            ppa = a + b
            div = d + e

In [13]:
for match in report_long_matches(X, 3):
    print(match)

(4, [4], [5])
(4, [0], [7])
(5, [4], [1, 6])
(5, [3], [2])


In [14]:
def display_long_matches(X, L):
    from IPython.display import display_html
    html = '<pre style="line-height: 100%">'
    for match in report_long_matches(X, L):
        k, ma, mb = match
        for i in sorted(ma):
            for j in sorted(mb):
                html += 'match ending at position %s between haplotypes %s and %s:<br/><br/>' % (k, i, j)
                h1 = X[i][k-L:k+1]
                h2 = X[j][k-L:k+1]
                html += str(i) + '|' + ''.join(str(allele) for allele in h1[:-1]) 
                html += str(h1[-1]) + '<br/>'
                html += str(j) + '|<strong><u>' + ''.join(str(allele) for allele in h2[:-1]) + '</u></strong>'
                html += str(h2[-1]) + '<br/><br/>'
    html += '</pre>'
    display_html(html, raw=True)

In [15]:
display_long_matches(X, 3)

match ending at position 4 between haplotypes 4 and 5: 4|0000 5| 000 1 match ending at position 4 between haplotypes 0 and 7: 0|1010 7| 101 1 match ending at position 5 between haplotypes 4 and 1: 4|0000 1| 000 1 match ending at position 5 between haplotypes 4 and 6: 4|0000 6| 000 1 match ending at position 5 between haplotypes 3 and 2: 3|1110 2| 111 1

In [16]:
def report_set_maximal_matches(X):

    # M haplotypes
    M = len(X)

    # N variable sites
    N = len(X[0])
    
    # initialise positional prefix array
    ppa = list(range(M))
    
    # initialise divergence array
    div = [0] * M
    
    # iterate over variants
    for k in range(N):
        
        # sentinel
        div.append(k+1)
        
        for i in range(M):
            
            m = i - 1
            n = i + 1
            match_continues = False
            
            if div[i] <= div[i+1]:
                # match to previous neighbour is longer, scan "down" the haplotypes (decreasing indices)
                while div[m+1] <= div[i]:
                    if X[ppa[m]][k] == X[ppa[i]][k]:
                        match_continues = True
                        break
                    m -= 1
            if match_continues:
                continue
                    
            if div[i] >= div[i+1]:
                # match to next neighbour is longer, scan "up" the haplotypes (increasing indices)
                while div[n] <= div[i+1]:
                    if X[ppa[n]][k] == X[ppa[i]][k]:
                        match_continues = True
                        break
                    n += 1
            if match_continues:
                continue
                
            for j in range(m+1, i):
                if div[i] < k:  # N.B., avoid 0 length matches, not in Durbin (2014)
                    yield ppa[i], ppa[j], div[i], k
                
            for j in range(i+1, n):
                if div[i+1] < k:  # N.B., avoid 0 length matches, not in Durbin (2014)
                    yield ppa[i], ppa[j], div[i+1], k
                    
        # build next prefix and divergence arrays
        if k < N - 1:        
                
            # setup intermediates
            a = list()
            b = list()
            d = list()
            e = list()
            p = q = k + 1

            # iterate over haplotypes in prefix sorted order
            for index, match_start in zip(ppa, div):

                # current haplotype
                haplotype = X[index]

                # allele for current haplotype
                allele = haplotype[k]

                # update intermediates
                if match_start > p:
                    p = match_start
                if match_start > q:
                    q = match_start

                # update intermediates
                if allele == 0:
                    a.append(index)
                    d.append(p)
                    p = 0
                else:
                    b.append(index)
                    e.append(q)
                    q = 0

            # construct the new arrays for k+1
            ppa = a + b
            div = d + e


In [17]:
for match in report_set_maximal_matches(X):
    print(match)


(4, 0, 0, 1)
(4, 3, 0, 1)
(4, 7, 0, 1)
(5, 1, 0, 1)
(5, 2, 0, 1)
(5, 6, 0, 1)
(3, 0, 0, 2)
(3, 7, 0, 2)
(2, 1, 0, 2)
(2, 6, 0, 2)
(4, 5, 1, 4)
(5, 4, 1, 4)
(0, 7, 0, 4)
(7, 0, 0, 4)
(4, 1, 2, 5)
(4, 6, 2, 5)
(3, 2, 1, 5)
(2, 3, 1, 5)


In [18]:
def display_set_maximal_matches(X):
    from IPython.display import display_html
    from operator import itemgetter
    from itertools import groupby
    html = '<pre style="line-height: 100%">'
    matches = sorted(report_set_maximal_matches(X), key=itemgetter(0, 2))
    for i, sub_matches in groupby(matches, key=itemgetter(0)):
        html += 'set maximal matches for haplotype %s:<br/><br/>' % i
        hi = X[i]
        html += str(i) + '|' + ''.join(map(str, hi)) + '<br/>'
        for _, j, k1, k2 in sub_matches:
            hj = X[j]
            html += str(j) + '|' + ''.join(map(str, hj[:k1]))
            html += '<strong><u>' + ''.join(map(str, hj[k1:k2])) + '</u></strong>'
            html += ''.join(map(str, hj[k2:])) + '<br/>'
        html += '<br/>'
    html += '</pre>'
    display_html(html, raw=True)

In [19]:
display_set_maximal_matches(X)

set maximal matches for haplotype 0: 0|010101 7| 0101 10 set maximal matches for haplotype 2: 2|111111 1| 11 0001 6| 11 0001 3|0 1111 0 set maximal matches for haplotype 3: 3|011110 0| 01 0101 7| 01 0110 2|1 1111 1 set maximal matches for haplotype 4: 4|000000 0| 0 10101 3| 0 11110 7| 0 10110 5|1 000 10 1|11 000 1 6|11 000 1 set maximal matches for haplotype 5: 5|100010 1| 1 10001 2| 1 11111 6| 1 10001 4|0 000 00 set maximal matches for haplotype 7: 7|010110 0| 0101 01

In [20]:
import random
M = 3
N = 30
Y = [[random.randint(0, 1) for _ in range(N)] for _ in range(M)]
display_set_maximal_matches(Y)


set maximal matches for haplotype 0: 0|001111011001010100111100111000 1| 0011 01100101001100010111000110 2|1010 1101 0111000001010111000000 1|0011011001 010 01100010111000110 2|10101101011100 0 001010111000000 1|001101100101001 100 010111000110 1|0011011001010011000 1 0111000110 2|1010110101110000010 1 0111000000 1|001101100101001100010 1 11000110 2|101011010111000001010 1 11000000 set maximal matches for haplotype 1: 1|001101100101001100010111000110 0| 0011 11011001010100111100111000 2|10101 1 010111000001010111000000 0|00111 1 011001010100111100111000 2|10101101 01 11000001010111000000 0|0011110110 010 10100111100111000 2|10101101011 100 0001010111000000 0|001111011001010 100 111100111000 2|101011010111000001 010111000 000 set maximal matches for haplotype 2: 2|101011010111000001010111000000 0|0 01 111011001010100111100111000 1|0 01 101100101001100010111000110 0|0011 1101 1001010100111100111000 1|00110110 01 01001100010111000110 1|00110110010 100 1100010111000110 0|00111101100101 0 100111100111000 0|0011110110010101 0 0111100111000 1|0011011001010011 0 0010111000110 1|001101100101001100 010111000 110